In [1]:
!pip install pymupdf pandas
!pip install pdfplumber
!pip install python-docx

In [2]:
import fitz  #PyMuPDF
import pandas as pd
from IPython.display import display, HTML

import pdfplumber
import pandas as pd
from IPython.display import display, HTML

In [3]:
def run_final_analysis(pdf_path):
    doc = fitz.open(pdf_path)
    page_data = []
    full_text_list = []

    for i, page in enumerate(doc):
        # 텍스트 추출
        text = page.get_text("text") 
        clean_text = " ".join(text.split()).strip()
        
        # 레이아웃 분석
        d = page.get_text("dict")
        blocks = [b for b in d.get("blocks", []) if b.get("type") == 0]
        spans_count = sum(len(line.get("spans", [])) for b in blocks for line in b.get("lines", []))

        # 데이터 저장
        ocr_needed = (len(clean_text) < 30 or spans_count < 3)
        page_data.append({
            "Page": i + 1,
            "Text_Len": len(clean_text),
            "Spans": spans_count,
            "OCR_Needed": "YES" if ocr_needed else "NO"
        })
        full_text_list.append(text) # 원본 줄바꿈 포함 텍스트 저장

    doc.close()

    # 1. 지표 요약 출력
    df = pd.DataFrame(page_data)
    display(HTML("<h3>[1] 파싱 지표 요약</h3>"))
    display(df)

    # 2. 파싱된 텍스트 원본 출력 (이 부분이 핵심입니다)
    display(HTML("<h3>[2] 페이지별 텍스트 추출 결과 (Raw Text)</h3>"))
    for i, content in enumerate(full_text_list):
        print(f"\n--- PAGE {i+1} START ---")
        if content.strip():
            print(content.strip())
        else:
            print("⚠️ 이 페이지에는 추출된 텍스트가 없습니다.")
        print(f"--- PAGE {i+1} END ---\n" + "="*50)
    
    return full_text_list # 나중에 LLM에 넘길 수 있도록 리스트 반환

# 실행 예시 (가장 결과가 궁금한 파일을 넣어보세요)
extracted_texts = run_final_analysis("test_file/test_irony.pdf")

,Page,Text_Len,Spans,OCR_Needed
0,1,691,102,NO



--- PAGE 1 START ---
VI. 이사회 등 회사의 기관에 관한 사항
 
1. 이사회에 관한 사항
 
가. 이사회 구성 개요 
사업보고서 제출일 현재 당사의 이사회는 3명의 상근이사, 1명의 사외이사 총 4명의 이사로
구성되어 있습니다. 
⇒ 각 이사의 주요 이력 및 업무분장은 VIII. 임원 및 직원 등에 관한 사항을 참조하시기 바랍
니다. 
 
  
(1) 사외이사 및 그 변동현황
 
※ 이사의 수는 사외이사를 포함한 총 이사의 수입니다. 
  
나. 주요 의결사항
 
(단위 : 명)
이사의 수
사외이사 수
사외이사 변동현황
선임
해임
중도퇴임
4
1
-
-
-
회
차
개최일
자
의 안 내 용
가
결
 여
부
사내
이사
사내
이사
사외
이사
안 재
일
 (출
석률:
 100
%)
안 갑
원
 (출
석률:
 100
%)
김 재
호
 (출
석률:
 100
%)
1
2024.0
1.19
신규채무약정 1건에 관한 건
가
결
찬성
찬성
찬성
2
2024.0
2.05
내부회계관리제도 운영실태 보고의 건
가
결
찬성
찬성
찬성
3
2024.0
2.08
내부회계관리제도 운영실태 평가 감사보고의 건
가
결
찬성
찬성
찬성
4
2024.0
2.26
제44기(2023년도) 현금배당 결정의 건
- 배당금액 : 4,189,282,050원
- 배당주식수 : 28,600,000주중 자기주식 보유분
671,453주를 제외한 27,928,547주
- 배 당 금 : 1주당 150원의 현금배당 예정
가
결
찬성
찬성
찬성
--- PAGE 1 END ---


In [4]:
# 쫑 버전
def handle_pdf(path: str, ext: str):
    PDF_PAGE_MIN_CHARS = 30 # 어제 30자로 정해서 수정했습니다.
    PDF_PASS_RATIO = 0.7

    with pdfplumber.open(path) as pdf:
        total_pages = len(pdf.pages)
        valid_page_count = 0
        pages_data = {}  # 저희 .json포맷

        for i, page in enumerate(pdf.pages):
            # 1. 텍스트 추출 및 정제 (split/join으로 공백/제어문자 해결해야 글자수 허수 없음)
            t = page.extract_text() or ""
            clean_t = " ".join(t.split()).strip() 
            
            # 2. 지표 체크
            if len(clean_t) >= PDF_PAGE_MIN_CHARS:
                valid_page_count += 1
            
            # 3. 기존 포맷 구조 생성
            pages_data[f"page{i+1}"] = {
                "text": clean_t,
                "tableCount": 0,
                "tables": {}
            }

        pass_ratio = valid_page_count / max(total_pages, 1)

    # 최종 분기
    if pass_ratio >= PDF_PASS_RATIO:
        # [TEXT 모드]
        return {
            "kind": "PDF",
            "ext": ext,
            "mode": "text",
            "pass_ratio": pass_ratio,
            "content": {
                "pageCount": total_pages,
                "pages": pages_data
            }
        }
    else:
        # [OCR 모드] : cloverocr 결과도 반드시 저희가 정한 json 구조여야 합니다.
        ocr_result = cloverocr(path)
        return {
				"kind": "PDF", 
				"ext": ext, 
				"mode": "ocr", 
				"pass_ratio": pass_ratio,
                "content": {
                "pageCount": len(ocr_result.get("pages", {})),
                "pages": ocr_result.get("pages", {})
            }
        }
       

In [5]:
from pathlib import Path
import pandas as pd
from docx import Document
import pdfplumber 


CSV_EXT = {"csv"}
EXCEL_EXT = {"xls", "xlsx"}
IMAGE_EXT = {"jpg", "png"}
OTHER_EXT = {"pdf", "docx"}

def handle_file(path: str):
    ext = Path(path).suffix.lower().lstrip(".")

    # CSV
    if ext in CSV_EXT:
        for enc in ["cp949", "euc-kr", "utf-8"]:
            try:
                df = pd.read_csv(path, encoding=enc)
                return {"kind": "CSV", "ext": ext, "dataframe": df}
            except UnicodeDecodeError: continue
        raise ValueError("인코딩 실패: cp949, euc-kr, utf-8 encoding을 지원합니다")

    # EXCEL
    if ext in EXCEL_EXT:
        df = pd.read_excel(path)
        return {"kind": "EXCEL", "ext": ext, "dataframe": df}

    # IMAGE
    if ext in IMAGE_EXT:
        ocr_result = cloverocr(path)
        return {"kind": "IMAGE", "ext": ext, "content": ocr_result}

    # PDF
   


    # DOCX
    if ext == "docx":
        doc = Document(path)

        paragraphs = []
        for i, p in enumerate(doc.paragraphs):
                text = p.text.strip()
                if text:
                    paragraphs.append({
                        "index": i,
                        "text": text
                    })

        return {"kind": "DOCX", "ext": ext, "content": {"paragraph_count": len(paragraphs), "paragraphs": paragraphs}}

    raise ValueError(f"지원하지 않는 파일 형식입니다: {ext}")

In [7]:
import os
import pandas as pd
from IPython.display import display

# ### 테스트 대상 PDF 리스트 (이미지 트리 구조 반영)
pdf_test_samples = [
    "test_file/test.pdf", 
    "test_file/sample.pdf", 
    "test_file/sample2.pdf",
    "test_file/test_irony.pdf", 
    "test_file/test_table.pdf"
]

def verify_handle_pdf_only():
    all_results = []

    print(f"🚀 [handle_pdf] 단독 로직 검증 시작 (쫑 버전)")
    print("=" * 75)

    for path in pdf_test_samples:
        if not os.path.exists(path):
            print(f"⚠️ 파일 없음: {path}")
            continue

        try:
            # 1. handle_pdf 직접 호출 (ext는 "pdf"로 고정)
            res = handle_pdf(path, "pdf")

            # 2. 결과 데이터 추출
            mode = res.get("mode", "N/A")
            pass_ratio = res.get("pass_ratio", 0.0)
            content = res.get("content", {})
            page_count = content.get("pageCount", 0)
            pages = content.get("pages", {})

            # 3. '쫑 버전' 스키마(page1 -> text) 접근 테스트
            first_page_text = pages.get("page1", {}).get("text", "")
            preview = " ".join(first_page_text.split())[:50].strip()

            all_results.append({
                "파일명": path,
                "판정모드": mode.upper(),
                "통과비율(Ratio)": f"{pass_ratio:.2%}",
                "페이지수": page_count,
                "첫페이지_미리보기": f"{preview}..." if preview else "(데이터 없음)"
            })

            # 콘솔 로그 출력
            status_icon = "✅" if mode == "text" else "🚀"
            print(f"{status_icon} {path:25} | Mode: {mode.upper():4} | Ratio: {pass_ratio:6.2%} | Pages: {page_count}")

        except Exception as e:
            print(f"❌ {path} 처리 중 예외 발생: {str(e)}")

    print("\n" + "=" * 75)
    print("📊 [handle_pdf] 검증 결과 요약")
    print("=" * 75)

    if all_results:
        df = pd.DataFrame(all_results)
        display(df)
    else:
        print("검증된 결과가 없습니다. 파일 경로를 다시 확인해 주세요.")

# 실행
verify_handle_pdf_only()

🚀 [handle_pdf] 단독 로직 검증 시작 (쫑 버전)
❌ test_file/test.pdf 처리 중 예외 발생: name 'cloverocr' is not defined
❌ test_file/sample.pdf 처리 중 예외 발생: name 'cloverocr' is not defined
✅ test_file/sample2.pdf     | Mode: TEXT | Ratio: 100.00% | Pages: 10
✅ test_file/test_irony.pdf  | Mode: TEXT | Ratio: 100.00% | Pages: 1
✅ test_file/test_table.pdf  | Mode: TEXT | Ratio: 100.00% | Pages: 1

📊 [handle_pdf] 검증 결과 요약


,파일명,판정모드,통과비율(Ratio),페이지수,첫페이지_미리보기
0,test_file/sample2.pdf,TEXT,100.00%,10,Lorem ipsum pdf – 10 pages Lorem ipsum dolor s...
1,test_file/test_irony.pdf,TEXT,100.00%,1,VI. 이사회 등 회사의 기관에 관한 사항 1. 이사회에 관한 사항 가. 이사회 구...
2,test_file/test_table.pdf,TEXT,100.00%,1,"11. 정보보안 영역에서 보안사고 대응 및 백업 정책이 미충족이며, 백업 주기 데이..."
